### Prepare Fannie Mae Single Family mortgage dataset

**Intoduction**  
The initial population of mortgage loans in the primary dataset included Fannie Mae acquisitions between January 1, 2000 and March 31, 2012 with corresponding monthly performance data as of December 31, 2012.

The compressed dataset is about 50GB.

**Main steps**
- Download data from https://datadynamics.fanniemae.com/data-dynamics/#/reportMenu;category=HP
- Join datasets
- Create default flag

**Main references**
- Introduction: https://capitalmarkets.fanniemae.com/credit-risk-transfer/single-family-credit-risk-transfer/fannie-mae-single-family-loan-performance-data
- Data dictionary: https://capitalmarkets.fanniemae.com/media/20926/display
- Data preparation: https://github.com/ferrarisf50/Fannie-Mae-single-family-mortgage-loan-data/blob/master/Predict_default.ipynb


#### A. Import dataset

In [12]:
import os, glob, csv
from pyspark.sql import SparkSession
from tqdm import tqdm
import pandas as pd

from pyspark.sql.functions import col, when
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Initialize a Spark session and set options
spark = SparkSession.builder.appName("GetFannieMaeDataset").getOrCreate()

In [2]:
headers = [
    "POOL_ID",
    "LOAN_ID",
    "ACT_PERIOD",
    "CHANNEL",
    "SELLER",
    "SERVICER",
    "MASTER_SERVICER",
    "ORIG_RATE",
    "CURR_RATE",
    "ORIG_UPB",
    "ISSUANCE_UPB",
    "CURRENT_UPB",
    "ORIG_TERM",
    "ORIG_DATE",
    "FIRST_PAY",
    "LOAN_AGE",
    "REM_MONTHS",
    "ADJ_REM_MONTHS",
    "MATR_DT",
    "OLTV",
    "OCLTV",
    "NUM_BO",
    "DTI",
    "CSCORE_B",
    "CSCORE_C",
    "FIRST_FLAG",
    "PURPOSE",
    "PROP",
    "NO_UNITS",
    "OCC_STAT",
    "STATE",
    "MSA",
    "ZIP",
    "MI_PCT",
    "PRODUCT",
    "PPMT_FLG",
    "IO",
    "FIRST_PAY_IO",
    "MNTHS_TO_AMTZ_IO",
    "DLQ_STATUS",
    "PMT_HISTORY",
    "MOD_FLAG",
    "MI_CANCEL_FLAG",
    "Zero_Bal_Code",
    "ZB_DTE",
    "LAST_UPB",
    "RPRCH_DTE",
    "CURR_SCHD_PRNCPL",
    "TOT_SCHD_PRNCPL",
    "UNSCHD_PRNCPL_CURR",
    "LAST_PAID_INSTALLMENT_DATE",
    "FORECLOSURE_DATE",
    "DISPOSITION_DATE",
    "FORECLOSURE_COSTS",
    "PROPERTY_PRESERVATION_AND_REPAIR_COSTS",
    "ASSET_RECOVERY_COSTS",
    "MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS",
    "ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY",
    "NET_SALES_PROCEEDS",
    "CREDIT_ENHANCEMENT_PROCEEDS",
    "REPURCHASES_MAKE_WHOLE_PROCEEDS",
    "OTHER_FORECLOSURE_PROCEEDS",
    "NON_INTEREST_BEARING_UPB",
    "PRINCIPAL_FORGIVENESS_AMOUNT",
    "ORIGINAL_LIST_START_DATE",
    "ORIGINAL_LIST_PRICE",
    "CURRENT_LIST_START_DATE",
    "CURRENT_LIST_PRICE",
    "ISSUE_SCOREB",
    "ISSUE_SCOREC",
    "CURR_SCOREB",
    "CURR_SCOREC",
    "MI_TYPE",
    "SERV_IND",
    "CURRENT_PERIOD_MODIFICATION_LOSS_AMOUNT",
    "CUMULATIVE_MODIFICATION_LOSS_AMOUNT",
    "CURRENT_PERIOD_CREDIT_EVENT_NET_GAIN_OR_LOSS",
    "CUMULATIVE_CREDIT_EVENT_NET_GAIN_OR_LOSS",
    "HOMEREADY_PROGRAM_INDICATOR",
    "FORECLOSURE_PRINCIPAL_WRITE_OFF_AMOUNT",
    "RELOCATION_MORTGAGE_INDICATOR",
    "ZERO_BALANCE_CODE_CHANGE_DATE",
    "LOAN_HOLDBACK_INDICATOR",
    "LOAN_HOLDBACK_EFFECTIVE_DATE",
    "DELINQUENT_ACCRUED_INTEREST",
    "PROPERTY_INSPECTION_WAIVER_INDICATOR",
    "HIGH_BALANCE_LOAN_INDICATOR",
    "ARM_5_YR_INDICATOR",
    "ARM_PRODUCT_TYPE",
    "MONTHS_UNTIL_FIRST_PAYMENT_RESET",
    "MONTHS_BETWEEN_SUBSEQUENT_PAYMENT_RESET",
    "INTEREST_RATE_CHANGE_DATE",
    "PAYMENT_CHANGE_DATE",
    "ARM_INDEX",
    "ARM_CAP_STRUCTURE",
    "INITIAL_INTEREST_RATE_CAP",
    "PERIODIC_INTEREST_RATE_CAP",
    "LIFETIME_INTEREST_RATE_CAP",
    "MARGIN",
    "BALLOON_INDICATOR",
    "PLAN_NUMBER",
    "FORBEARANCE_INDICATOR",
    "HIGH_LOAN_TO_VALUE_HLTV_REFINANCE_OPTION_INDICATOR",
    "DEAL_NAME",
    "RE_PROCS_FLAG",
    "ADR_TYPE",
    "ADR_COUNT",
    "ADR_UPB",
    # "PAYMENT_DEFERRAL_MOD_EVENT_FLAG",
    # "INTEREST_BEARING_UPB",
]

len(headers)

108

In [13]:
# Get list of CSV files
csv_files = glob.glob("C:\\projects\\risk_datasets\\data\\*.csv")

# Output path
output_path = "C:\\projects\\risk_datasets\\final_result.csv"

# Write the headers to the output file
with open(output_path, 'w', newline='') as f_out:
    writer = csv.writer(f_out, delimiter='|')
    writer.writerow(headers)

# Function to read a CSV file and append its content to the output file
def append_csv_to_output(file_path, output_path):
    with open(file_path, 'r') as f_in:
        reader = csv.reader(f_in, delimiter='|')
        with open(output_path, 'a', newline='') as f_out:
            writer = csv.writer(f_out, delimiter='|')
            for row in reader:
                writer.writerow(row)

# Loop through the CSV files and append them to the output CSV
for file in tqdm(csv_files, desc="Processing CSV files"):
    append_csv_to_output(file, output_path)

Processing CSV files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [09:15<00:00, 138.99s/it]


#### B. Get a sample from the dataset

In [17]:
# Import the complete dataset
csv_path = "C:\\projects\\risk_datasets\\final_result.csv"
data = spark.read.csv(csv_path, sep='|', header=True, inferSchema=True)

# Clean the dataset
# data["ACT_PERIOD_V2"] = data["ACT_PERIOD"].astype(str).apply(custom_date_parser)

data.limit(10).show()

+-------+------------+----------+-------+------------------+--------------------+---------------+---------+---------+--------+------------+-----------+---------+---------+---------+--------+----------+--------------+-------+----+-----+------+---+--------+--------+----------+-------+----+--------+--------+-----+-----+---+------+-------+--------+---+------------+----------------+----------+-----------+--------+--------------+-------------+------+--------+---------+----------------+---------------+------------------+--------------------------+----------------+----------------+-----------------+--------------------------------------+--------------------+------------------------------------------+-------------------------------------+------------------+---------------------------+-------------------------------+--------------------------+------------------------+----------------------------+------------------------+-------------------+-----------------------+------------------+------------+

#### B. Append default flag

Zero_Bal_Code values:
- 01 = Prepaid or Matured
- 02 = Third Party Sale
- 03 = Short Sale06 = Repurchased
- 09 = Deed-in-Lieu; REO Disposition
- 15 = Non Performing Note Sale
- 16 = Reperforming Note Sale
- 96 = Removal (non-credit event)Applies to all CAS deals prior to and including 2015-C03:
- 97 = Delinquency (credit event due to D180)
- 98 = Other Credit Event

In [ ]:
# Create a default flag
data['Default']=data.apply(lambda row : 1 
     if row['Zero_Bal_Code'] in ['02','03','09','15'] and str(row['DISP_DT']) != 'NaT' 
     else (0 if row['Zero_Bal_Code'] in ['01','06','16'] else np.nan),axis=1)
data['Default'].value_counts(dropna=False)

In [ ]:
# Add a new boolean column based on the conditions
data = data.withColumn("is_first_credit_event",
                       (col("Zero.Bal.Code").isin(["03", "09"])) | 
                       ((col("Delq.Status") < 999) & (col("Delq.Status") >= 6)))

In [ ]:
# Select relevant columns and add new ones
data = data.withColumn("FCE_DTE", col("Monthly.Rpt.Prd")) \
           .withColumn("FCE_UPB", col("LAST_UPB")) \
           .withColumn("SPDelq1", col("Delq.Status")) \
           .withColumn("CountFC", col("Count"))

# Define window specification
window_spec = Window.partitionBy("LOAN_ID").orderBy("Monthly.Rpt.Prd")

# Select the first entry for each LOAN_ID
first_ce = data.withColumn("row_num", row_number().over(window_spec))

#### C. Review the final dataset

In [ ]:
data['Zero_Bal_Code'].value_counts(dropna=False)

In [ ]:
data['Default'].value_counts(dropna=False)